In [1]:
# Python calculation for the example and exercise from Lecture 4 notes
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
import scipy.interpolate
import statistics
from statistics import NormalDist

# Problem Set 1 Question 9

In [ ]:
mu = [0.000356, 0.000267, 0.000133, 0.000153]
sigma = [[0.00007, 0.0001, -0.000045, 0.000068],
         [0.0001, 0.0004, -0.00008, 0.000241],
         [-0.000045, -0.00008, 0.000178, -0.000118],
         [0.000068, 0.000241, -0.000118, 0.000324]
        ]

mu10d = np.multiply(mu, 10)
sigma10d = np.multiply(sigma, 10)
w = np.array([1000000, 1000000, 1000000, 400000])
meanL = np.inner(w, mu10d) + 2500 * (0.001 * 10) / 0.01 
varianceL = np.dot(np.dot(w, sigma10d), np.transpose(w)) + 2500 * 2500 * 0.0005 * 0.0005 * 10

print(f"The mean is equal to: {meanL:,.0f}")
print(f"The variance is equal to: {varianceL:,.0f}")
print(f"The standard deviation is equal to: {np.sqrt(varianceL):,.0f}")

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
var = np.abs(stat.norm.ppf(0.01, loc=meanL, scale=np.sqrt(varianceL)))
print(f"The 10-day 99% VaR using the parametric VaR model is equal to: {var:,.0f}")

The mean is equal to: 10,672
The variance is equal to: 8,026,400,016
The standard deviation is equal to: 89,590
The 10-day 99% VaR using the parametric VaR model is equal to: 197,746


# Lesson 5 Example

In [3]:
file_path = ""
offset = 693594

df_appl = pd.read_csv(file_path + "AAPL.csv").set_index('Date')
df_appl.index = pd.to_datetime(df_appl.index).date
df_appl = df_appl.sort_index()
df_appl

,Close
2024-02-07,189.41
2024-02-08,188.32
2024-02-09,188.85
2024-02-12,187.15
2024-02-13,185.04
...,...
2025-02-03,228.01
2025-02-04,232.80
2025-02-05,232.47
2025-02-06,233.22


In [4]:
df_dbs = pd.read_csv(file_path + "DBS.csv").set_index('Date')
df_dbs.index = pd.to_datetime(df_dbs.index).date
df_dbs = df_dbs.sort_index()
df_dbs

,Close
2024-02-07,29.50
2024-02-08,29.51
2024-02-09,29.58
2024-02-13,29.54
2024-02-14,29.61
...,...
2025-02-03,44.31
2025-02-04,44.42
2025-02-05,44.32
2025-02-06,44.32


In [5]:
df_usdsgd = pd.read_csv(file_path + "USDSGD.csv").set_index('Date')
df_usdsgd.index = pd.to_datetime(df_usdsgd.index).date
df_usdsgd = df_usdsgd.sort_index()
df_usdsgd

,Close
2024-02-07,1.3434
2024-02-08,1.3472
2024-02-09,1.3461
2024-02-12,1.3445
2024-02-13,1.3511
...,...
2025-02-03,1.3604
2025-02-04,1.3527
2025-02-05,1.3484
2025-02-06,1.3506


# Interpolate the missing dates

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html

d1 = [ d.toordinal()-693594  for d in df_appl.index.tolist()]
p1 = df_appl['Close'].tolist()
interp1 =  scipy.interpolate.interp1d(d1, p1)

d2 = [ d.toordinal()-693594  for d in df_dbs.index.tolist()]
p2 = df_dbs['Close'].tolist()
interp2 =  scipy.interpolate.interp1d(d2, p2)

d3 = [ d.toordinal()-693594  for d in df_usdsgd.index.tolist()]
p3 = df_usdsgd['Close'].tolist()
interp3 =  scipy.interpolate.interp1d(d3, p3)

dlist = list(set(d1) | set(d2) | set(d3))

In [ ]:
# Note that this is using the interpolated data to calculate daily change (in percentage)

numchg = len(dlist)-1
appl = [ interp1(dlist[i+1]).flat[0] / interp1(dlist[i]).flat[0] - 1.0  for i in range(numchg) ]
dbs = [ interp2(dlist[i+1]).flat[0] / interp2(dlist[i]).flat[0] - 1.0  for i in range(numchg) ]
usdsgd = [ interp3(dlist[i+1]).flat[0] / interp3(dlist[i]).flat[0] - 1.0  for i in range(numchg) ]

# Calculate mean vector and covariance matrix

In [8]:
alldata = {'appl' : appl,
           'dbs' : dbs,
           'usdsgd' : usdsgd }

df_all = pd.DataFrame(alldata, columns=['appl', 'dbs', 'usdsgd'])

corr_mat = df_all.corr()
cov_mat = df_all.cov()

appl_mean = np.average(appl)
appl_std = statistics.stdev(appl)

dbs_mean = np.average(dbs)
dbs_std = statistics.stdev(dbs)

usdsgd_mean = np.average(usdsgd)
usdsgd_std = statistics.stdev(usdsgd)

mean_vec = np.array([appl_mean, dbs_mean, usdsgd_mean])

mean10d_vec = np.array([appl_mean*10, dbs_mean*10, usdsgd_mean*10])

cov10d_mat = (cov_mat * 10).to_numpy()

In [9]:
mean_vec

array([8.04165189e-04, 1.64027617e-03, 3.42693964e-05])

In [10]:
mean10d_vec

array([0.00804165, 0.01640276, 0.00034269])

In [11]:
cov10d_mat

array([[ 2.06432945e-03,  3.05983904e-04, -6.41512277e-05],
       [ 3.05983904e-04,  1.09789749e-03, -2.07773463e-05],
       [-6.41512277e-05, -2.07773463e-05,  8.57834970e-05]])

# Parametric VaR

In [12]:
# https://numpy.org/doc/stable/reference/generated/numpy.inner.html
# https://numpy.org/doc/stable/reference/generated/numpy.dot.html

usdsgd_p0 = 1.354   # Last price
appl_p0 = 227.63    # Last price
dbs_p0 = 44.68      # Last price
n_appl = 100000     # Number of shares of APPL
n_dbs = 200000      # Number of shares of DBS

In [13]:
# Weight vector
w_parametric = np.array([n_appl*appl_p0*usdsgd_p0, 
                         n_dbs*dbs_p0, 
                         n_appl*appl_p0*usdsgd_p0])

w_parametric

array([30821102.,  8936000., 30821102.])

In [14]:
# Calculating mean, variance and 1-percentile for 10 day
m10d = np.inner(w_parametric, mean10d_vec)
v10d = np.inner(np.dot(w_parametric, cov10d_mat), w_parametric)
var10d = np.abs(stat.norm.ppf(0.01, loc=m10d, scale=np.sqrt(v10d)))

In [15]:
# Calculating mean, variance and 1-percentile for 1 day
m1d =  np.inner(w_parametric, mean_vec)
v1d = np.inner(np.dot(w_parametric, cov_mat), w_parametric)
var1d = np.abs(stat.norm.ppf(0.01, loc=m1d, scale=np.sqrt(v1d)))

In [16]:
m1d

40498.98572272945

In [17]:
v1d

216537041521.535

In [18]:
var1d

1042033.0503516665

In [19]:
print("")
print("")
print("============================================================================================================================")
print(f"Parametric VaR [1d, 99%]: {var1d:,.0f}")
print(f"Parametric VaR [10d, 99%]: {var10d:,.0f}")
print("============================================================================================================================")



Parametric VaR [1d, 99%]: 1,042,033
Parametric VaR [10d, 99%]: 3,018,277


# Monte Carlo VaR

In [20]:
# full revaluation 1d pnl evaluation
def pnl1d_full(appl_daily_return, dbs_return, usdsgd_return):
    return n_appl*appl_p0*usdsgd_p0*((1+appl_daily_return)*(1+usdsgd_return)-1) + n_dbs*dbs_p0*((1+dbs_return)-1)

In [21]:
# sensitivity based 1d pnl evaluation 
def pnl1d_sen(appl_daily_return, dbs_return, usdsgd_return):
    return n_appl*appl_p0*usdsgd_p0*(appl_daily_return+usdsgd_return) + n_dbs*dbs_p0*dbs_return

In [22]:
# full revaluation 10d pnl evaluation
def pnl10d_full(returns):
    a10d = 0.0
    fx10d = 0.0
    d10d = 0.0
    for i in range(0,10):
        a10d = a10d + returns[i][0]
        fx10d = fx10d  +returns[i][2]
        d10d = d10d  +returns[i][1]
    return n_appl*appl_p0*usdsgd_p0*a10d + n_appl*appl_p0*usdsgd_p0*fx10d + n_dbs*dbs_p0*d10d

In [23]:
# sensitivity based 10d pnl evaluation 
def pnl10d_sen(returns):
    a10d = 1.0
    d10d = 1.0
    for i in range(0,10):
        a10d = a10d * (1+returns[i][0])*(1+returns[i][2])
        d10d = d10d * (1+returns[i][1])
    return n_appl*appl_p0*usdsgd_p0*(a10d-1)  +n_dbs*dbs_p0*(d10d-1)

In [24]:
n_mc = 100000

# The lower triangle matrix
factor_loadings = np.linalg.cholesky(corr_mat)

# Number of simulations, number of risk factors
uniforms = np.random.uniform(size=(n_mc,3))

# Convert to independent standard normal
snorms = [ [NormalDist().inv_cdf(u) for u in r]  for r in uniforms]

# Correlated standard normal
snorms_correlated = np.dot(snorms, factor_loadings.transpose())

# Add back the mean, scale by standard deviation
return1d_sample = [ [ appl_mean + appl_std * z[0],   
                     dbs_mean + dbs_std * z[1], 
                     usdsgd_mean + usdsgd_std * z[2] ]  for z in snorms_correlated]

In [25]:
# For each row, get 1 day full revaluation
pnl1d_full_sample = [ pnl1d_full(s[0], s[1], s[2])  for s in return1d_sample]
# Get percentile
var1d_full_mc = np.abs(np.percentile(pnl1d_full_sample, 1))

In [26]:
# Every scenario for 10 day VAR, you need 10 rows of data
pnl10d_full_sample = [ pnl10d_full(return1d_sample[k*10:(k+1)*10]) for k in range(0, int(n_mc/10)) ]
var10d_full_mc = np.abs(np.percentile(pnl10d_full_sample, 1))

In [27]:
pnl1d_sen_sample = [ pnl1d_sen(s[0], s[1], s[2])  for s in return1d_sample]
var1d_sen_mc = np.abs(np.percentile(pnl1d_sen_sample, 1))

In [28]:
pnl10d_sen_sample = [ pnl10d_sen(return1d_sample[k*10:(k+1)*10]) for k in range(0, int(n_mc/10)) ]
var10d_sen_mc = np.abs(np.percentile(pnl10d_sen_sample, 1))

In [29]:
print("")
print("")
print("============================================================================================================================")
print("Monte Carlo VaR:")
print(f"VaR [1d, 99%], Full Revaluation: {var1d_full_mc:,.0f}") 
print(f"VaR [10d, 99%], Full Revaluation: {var10d_full_mc:,.0f}") 
print(f"VaR [10d, 99%] using square-root-of-time rule: {np.sqrt(10)*var1d_full_mc:,.0f}") # Take 1 day VaR and multiply by sqrt of 10
print("")
print(f"VaR [1d, 99%], Sensitivity: {var1d_sen_mc:,.0f}") 
print(f"VaR [10d, 99%], Sensitivity: {var10d_sen_mc:,.0f}") 
print("============================================================================================================================")



Monte Carlo VaR:
VaR [1d, 99%], Full Revaluation: 1,045,374
VaR [10d, 99%], Full Revaluation: 3,059,640
VaR [10d, 99%] using square-root-of-time rule: 3,305,762

VaR [1d, 99%], Sensitivity: 1,045,764
VaR [10d, 99%], Sensitivity: 2,960,888


# Historical VaR

In [30]:
hist_returns = df_all.to_numpy().tolist()
hist_returns

[[-0.005754712000422435, 0.0003389830508475633, 0.00282864373976488],
 [0.0028143585386575243, 0.0023720772619448827, -0.0008165083135390772],
 [-0.009001853322742837, -0.0010141987829614951, -0.0011886189733304464],
 [-0.011274378840502308, -0.00033840947546526223, 0.004908888062476802],
 [-0.00480977086035439, 0.0023696682464455776, -0.002072385463696147],
 [-0.0015748031496062298, 0.023302938196555267, -0.002002521693985182],
 [-0.008430327423039286, 0.016171617161716112, 0.0010404280618312445],
 [-0.0030854039822281187, 0.007794738551477831, -0.0008166295471416785],
 [-0.0010316510543473267, 0.0009668063164678387, -0.0017088936770934815],
 [0.004185944040537404, -0.006439150032195751, 0.0002977076510868315],
 [0.011243966652040527, 0.0, -0.0007440476190476719],
 [-0.010034170418180777, -0.0035644847699286553, 0.00022338049143710847],
 [-0.007451238220469114, -0.009756097560975618, 0.0008188788803693559],
 [0.008114374034003058, 0.0, -0.0002975304968758641],
 [-0.006625417510814247,

In [31]:
pnl1d_full_hist_sample = [ pnl1d_full(s[0], s[1], s[2])  for s in hist_returns]
var1d_full_hist = np.abs(np.percentile(pnl1d_full_hist_sample, 1))

In [32]:
pnl1d_full_hist_sample

[-87657.2026035879,
 82702.00271852182,
 -322814.68757480953,
 -200921.25114745426,
 -190633.07111906784,
 98075.15856584617,
 -83525.60750924196,
 -50533.531883496325,
 -75772.8904124143,
 80689.25030646034,
 323361.22473152145,
 -334300.6765235547,
 -291785.1714616369,
 240849.32139869864,
 -168102.35153747402,
 -152992.2252261672,
 -163302.70360609633,
 -835314.5128503498,
 -898065.4887165803,
 -212625.14052828422,
 -105536.97117789584,
 267073.4058240565,
 303375.3795543856,
 98201.49565621078,
 -256246.36881194817,
 602912.6359049785,
 -16671.459731175404,
 290722.6520012098,
 503742.03998445446,
 388510.18183040427,
 -980889.6484095216,
 322603.07303126925,
 -403849.8947713257,
 -67988.43604935164,
 839138.0267118535,
 -416465.2929573922,
 -92476.54437452686,
 -148265.19349764936,
 -171708.59950750918,
 49653.56773433123,
 -86769.45309331453,
 -55031.48715719348,
 -216501.5946993113,
 219545.82464734936,
 -113265.23291887816,
 1343044.888953573,
 484305.72884863213,
 -758150.6872

In [33]:
var1d_full_hist

1027654.9873520167

In [34]:
pnl1d_sen_hist_sample = [ pnl1d_sen(s[0], s[1], s[2])  for s in hist_returns]
var1d_sen_hist = np.abs(np.percentile(pnl1d_sen_hist_sample, 1))

In [35]:
pnl1d_sen_hist_sample

[-87155.49577831528,
 82772.8279818381,
 -323144.4663899928,
 -199215.4676223411,
 -190940.28659326897,
 97977.96183295292,
 -83255.2710244327,
 -50611.18972012063,
 -75827.22745934316,
 80650.8414323219,
 323619.0755076142,
 -334231.59293682163,
 -291597.1115267455,
 240923.73197583805,
 -167783.2848673272,
 -153017.5929635648,
 -163523.67487694605,
 -835780.0857642425,
 -898522.2445004127,
 -213124.38759724944,
 -105631.69993853854,
 267475.7161482038,
 303649.2521142196,
 98085.63687107564,
 -256246.36881194817,
 601977.3092413084,
 -16590.38244925226,
 290502.402600907,
 502866.54156513006,
 389287.2257750201,
 -976941.4993845328,
 321981.9318871546,
 -404399.9009652366,
 -68049.55177355328,
 838117.6828732209,
 -415861.59124112455,
 -92519.97410204847,
 -147829.6858998071,
 -171900.01530068315,
 49992.827250188115,
 -86735.89449957,
 -55072.65056445822,
 -216669.10028855738,
 220009.62813186104,
 -110967.55200833369,
 1343734.4744578332,
 482694.4460761816,
 -757358.6714024787,
 -

In [36]:
var1d_sen_hist

1028280.2340302861

In [37]:
print("")
print("")
print("============================================================================================================================")
print("Historical VaR:")
print(f"VaR [1d, 99%], Full Revaluation: {var1d_full_hist:,.0f}") 
print(f"VaR [10d, 99%] using square-root-of-time rule: {np.sqrt(10)*var1d_full_hist:,.0f}")
print("")
print(f"VaR [1d, 99%], Sensitivity: {var1d_sen_hist:,.0f}") 
print("============================================================================================================================")



Historical VaR:
VaR [1d, 99%], Full Revaluation: 1,027,655
VaR [10d, 99%] using square-root-of-time rule: 3,249,730

VaR [1d, 99%], Sensitivity: 1,028,280
